# Creation of Inital Decision Tree Model

Tree models where the target variable can take a discrete set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. 

In [ ]:
# Import relevant libraries and set display options 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline 
pd.set_option("display.max_rows", 10)

In [ ]:
# Create dataframe including all covariables to explore for initial model 
df = pd.read_csv('CompleteBaselineAcuteDataVoxtox.csv')
recurrence_df = pd.read_csv('RecurrencePatientIDs.csv')
df = pd.merge(df, recurrence_df, how='outer', on=['PATIENT_ID'])

In [ ]:
df['LOCAL_RECURRENCE'].fillna(0, inplace=True)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df

## One Hot Encoding for Cateogorical Variables 
- This replaces categorical string values with numerical ones which is easier for ML algorithms to interpret 

In [ ]:
# Functions for one hot encoding 
def binary_sex(sex):
    if sex == "M":
        new_sex = 0 
    else: 
        new_sex = 1
    
    return new_sex
        
    
def binary_definitive(definitive_rt):
    if definitive_rt == 'Definitive':
        new_defin = 1
    else:
        new_defin = 0
        
    return new_defin


def site_categories(primary_site):
    if primary_site == 'oropharynx':
        site = '1'
    elif primary_site == 'oral cavity':
        site = '2'
    elif primary_site == 'hypopharynx':
        site = '3'
    elif primary_site == 'larynx':
        site = '4'
    elif primary_site == 'nasopharynx':
        site = '5'
    else: 
        site = "6"
    return site 

df['BINARY_SEX'] = df.apply(lambda x: binary_sex(x['SEX']),axis=1) 
df['BINARY_DEFINITIVE'] = df.apply(lambda x: binary_definitive(x['DEFINITIVE_RT']),axis=1) 
df['CATEGORICAL_SITE'] = df.apply(lambda x: site_categories(x['PRIMARY_SITE']),axis=1) 

In [ ]:
df

## Check Correlations Between Variables
Use a correlation matrix that uses both size and gradient to determine the correlations between the variables selected from the voxtox data
- Max Dose is highly correlated with BED and Definitive RT - so I will just keep maxdose 
- High correlation between the peak values and STATScore - just keep STATscore

In [ ]:
from heatmap import heatmap, corrplot
    
check_corr = df[['HEIGHT', 'WEIGHT', 'SMOKER',
        'ALCOHOL', 'PRIMARY_SURGERY',
       'NEOADJUVANT_CHEMO', 'AGE', 'BINARY_SEX', 'FRACTIONS', 'MAX_DOSE', 'BED',
       'DYS_AUC', 'XERO_AUC', 'MUCO_AUC', 'DYS_PEAK', 'XERO_PEAK', 'MUCO_PEAK',
       'STAT_SCORE', 'CATEGORICAL_SITE', 'TNM_STAGE', 'BINARY_DEFINITIVE',
       'DRY_MOUTH_BASELINE', 'DYSPHAGIA_BASELINE', 'ORAL_MUCOSITIS_BASELINE', 'LOCAL_RECURRENCE']]
plt.figure(figsize=(10, 10))
corrplot(check_corr.corr())

## Creating Decision Tree Model 
- Create df for the predictive variables and the class to predict as x,y
- Impute the missing information 
    - iterative imputation models each feature as a function of other features e.g. regression problem 
    - Each feature is imputed sequentially, allowing prior imputed values to be used as part of the prediction
    - 
- Use the standard scaler to normalise values between 0-1 for all categories
    - This overcomes any differences between scales, ranges or measured units by standardising to (μ = 0, σ = 1)
    - These differences can create problems for machine learning models that rely on distance calculations
    - After scaling all variables should contribute equally to the model, avoiding biasing 
- Train, Test split for DT 

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import roc_curve, auc

In [ ]:
x = df[['HEIGHT', 'WEIGHT', 'SMOKER',
        'ALCOHOL', 'PRIMARY_SURGERY',
       'NEOADJUVANT_CHEMO', 'AGE', 'BINARY_SEX', 
        'FRACTIONS', 'MAX_DOSE', 
       'DYS_AUC', 'XERO_AUC', 'MUCO_AUC',
       'STAT_SCORE', 'CATEGORICAL_SITE', 'TNM_STAGE',
       'DRY_MOUTH_BASELINE', 'DYSPHAGIA_BASELINE', 
        'ORAL_MUCOSITIS_BASELINE']]

y = df[['LOCAL_RECURRENCE']]

In [ ]:
#Check the amount of missing data in each column 
for i in list(x.columns):
    # count number of rows with missing values
    n_miss = x[[i]].isnull().sum()
    perc = n_miss / x.shape[0] * 100
    print('> %s, Missing: %d (%.1f%%)' % (i, n_miss, perc))

In [ ]:
x['WEIGHT'].describe()

## Remove known values from the df and run the imputation 
- compare the imputed values and the true values that have been removed to determine the success of the imputation 
- do this for values close the mean of the cohort, and outlier values for full test 

In [ ]:
new_df = x.copy()
new_df['WEIGHT'] = new_df['WEIGHT'].replace(82.8, np.nan) #0
new_df['WEIGHT'] = new_df['WEIGHT'].replace(129.1, np.nan) #225
new_df['HEIGHT'] = new_df['HEIGHT'].replace(153, np.nan) #3
new_df['HEIGHT'] = new_df['HEIGHT'].replace(173.734065, np.nan) #228

In [ ]:
imputer = IterativeImputer(max_iter=100)
imputer.fit(new_df)
x_transform_new = imputer.transform(new_df)
print('Missing: %d' % sum(np.isnan(x_transform_new).flatten()))
imputed_test_x = pd.DataFrame(x_transform_new, columns=list(x.columns))

In [ ]:
imputed_test_x

## Run Imputation for the full df without removing any values 

In [ ]:
# Impute the data using the Iterative Imputer method

imputer = IterativeImputer(max_iter=10)
imputer.fit(x)
x_transform = imputer.transform(x)
print('Missing: %d' % sum(np.isnan(x_transform).flatten()))
imputed_x = pd.DataFrame(x_transform, columns=list(x.columns))
#imputed_x.to_csv('ImputedVoxToxData.csv')

In [ ]:
imputed_x

In [ ]:
imputed_x
imputed_df = pd.concat([imputed_x, y], axis=1)
y=list(y['LOCAL_RECURRENCE'])

## Train and Test the Decision Tree Model 

In [ ]:
# Functions to determine use the train, test split for the decision tree model and its metrics 

def decision_tree(imputed_x, y):
    predictions = {}
    error = []
    accuracy = []
    scaler = StandardScaler()

    for i in range(1,6): 
        x_train, x_test, y_train, y_test = train_test_split(imputed_x, y, test_size = 0.3)
        scaler.fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
        dt_model = DecisionTreeClassifier(criterion = "gini", max_depth=None, class_weight={0:0.05, 1:0.95})
        dt_model.fit(x_train, y_train)
        y_pred = dt_model.predict(x_test)
        error.append(np.mean(y_pred != y_test))
        accuracy.append(sklearn.metrics.accuracy_score(y_test, y_pred))
        print(sklearn.metrics.accuracy_score(y_test, y_pred))
        con_matrix = confusion_matrix(y_test, y_pred)
        class_report = classification_report(y_test, y_pred)
        print(con_matrix, class_report)
        predictions[i] = [y_test, y_pred]
        
    #return predictions, error
    return accuracy, predictions


def decision_tree_metrics(predictions):
    accuracy = []
    brier_score = []
    precision = []
    f1_score = []
    roc_auc = []
    
    
    for key in predictions.keys(): 
        con_matrix = confusion_matrix(predictions[key][0], predictions[key][1])
        class_report = classification_report(predictions[key][0], predictions[key][1])
        print(con_matrix, class_report)
        print(sklearn.metrics.accuracy_score(predictions[key][0], predictions[key][1]))
        print(sklearn.metrics.brier_score_loss(predictions[key][0], predictions[key][1]))
        
        accuracy.append(sklearn.metrics.accuracy_score(predictions[key][0], predictions[key][1]))
        brier_score.append(sklearn.metrics.brier_score_loss(predictions[key][0], predictions[key][1]))
        precision.append(sklearn.metrics.precision_score(predictions[key][0], predictions[key][1]))
        f1_score.append(sklearn.metrics.f1_score(predictions[key][0], predictions[key][1]))
        false_positive_rate, true_positive_rate, thresholds = roc_curve(predictions[key][0], predictions[key][1])
        auc_value = auc(false_positive_rate, true_positive_rate)
        roc_auc.append(auc_value)

    
    return accuracy, brier_score, precision, f1_score, roc_auc   

In [ ]:
# Run the decision tree functions, where the metrics for each model are printed 
accuracy, predictions = decision_tree(imputed_x, y)
accuracy, brier_score, precision, f1_score, roc_auc = decision_tree_metrics(predictions)
print(np.mean(accuracy), np.mean(brier_score), np.mean(precision), np.mean(roc_auc))

## DT Hyperparameter Optimisation 
1. Max Depth of the Tree, how many branches we can include in the model 
2. Max features that can be included for the model at each node 

Visualise the max depth and max features against the AUC metric to find the optimal value for each parameter 

In [ ]:
max_depths = np.linspace(1, 32, 32, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
    dt = DecisionTreeClassifier(max_depth=max_depth)
    dt.fit(x_train, y_train)
    train_pred = dt.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    # Add auc score to previous train results
    train_results.append(roc_auc)
    y_pred = dt.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    # Add auc score to previous test results
    test_results.append(roc_auc)
    
    
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_depths, train_results, 'b', label='Train AUC')
line2, = plt.plot(max_depths, test_results, 'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC Score')
plt.xlabel('Tree Depth')
plt.show()
plt.savefig('treedepthtest2.png')

In [ ]:
max_features = list(range(1, x.shape[1]))
train_results = []
test_results = []
for max_feature in max_features:
    dt = DecisionTreeClassifier(max_features=max_feature)
    dt.fit(x_train, y_train)
    train_pred = dt.predict(x_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = dt.predict(x_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
    
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_features, train_results, 'b', label='Train AUC')
line2, = plt.plot(max_features, test_results, 'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC Score')
plt.xlabel('Max Features')
plt.show()
plt.savefig('maxfeaturetest1.png')

## Use Optimised Hyperparameters and Standardised Scaling for final output 

In [ ]:
# Employ feature scaling to ensure variables lie within comparable ranges 
scaler = StandardScaler()
x_train, x_test, y_train, y_test = train_test_split(imputed_x, y, test_size = 0.3)
check_x_test1 = x_test
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# Run the DT model using the scaled data 
dt_model = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5, \
                                    max_features=10, splitter='best')

dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

In [ ]:
print(sklearn.metrics.confusion_matrix(y_test, y_pred))

In [ ]:
# Determine area under the curve 
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
# Visualise the final decision tree 
import sklearn.tree
plt.figure(figsize=(10, 10))
sklearn.tree.plot_tree(dt_model) 